In [1]:
#importing required libraries
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

D:\Anaconda\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
D:\Anaconda\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
D:\Anaconda\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
D:\Anaconda\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is depr

In [2]:
#reading the train and test sets
train = pd.read_csv("digit-recognizer/train.csv")
test = pd.read_csv("digit-recognizer/test.csv")

In [3]:
#shifting the label column into a target variable
train_Y = train.loc[:,'label']
train.drop(labels=['label'],axis=1,inplace=True)

In [4]:
#normalizing the values in train and test sets
train_X = train/255.0
test_X = test/255.0

In [5]:
#reshaping the train and test dataframes into 28x28x1 numpy images
train_X=train_X.values.reshape(-1,28,28,1)
test_X=test_X.values.reshape(-1,28,28,1)

In [36]:
#callback function to run till 100% training accuracy is reached
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get('acc')==1):
            print("\n100% train accuracy reached\n")
            self.model.stop_training=True

callback=myCallback()

In [37]:
#the model having 2 convolution layers of 3x3 filters and hidden layer having 256 neurons with 10 output neurons
my_model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(64,(3,3),activation='relu', input_shape=(28,28,1)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64,(3,3),activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256,activation='relu'),
    tf.keras.layers.Dense(10,activation=tf.nn.softmax)
])

In [38]:
#compiling the model with categorical loss calculator and printing the summary
my_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['acc'])
my_model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 26, 26, 64)        640       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 13, 13, 64)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 11, 11, 64)        36928     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 1600)              0         
_________________________________________________________________
dense_4 (Dense)              (None, 256)               409856    
_________________________________________________________________
dense_5 (Dense)              (None, 10)               

In [39]:
#fitting the model (NOTE: This will take a lot of time 5-10 mins)
my_model.fit(train_X,train_Y, epochs=100, callbacks=[callback])

Epoch 1/100
42000/42000 [==============================] - 5s 128us/sample - loss: 0.1328 - acc: 0.9586
Epoch 2/100
42000/42000 [==============================] - 5s 124us/sample - loss: 0.0433 - acc: 0.9868s - loss: 0.0
Epoch 3/100
42000/42000 [==============================] - 5s 124us/sample - loss: 0.0288 - acc: 0.9905
Epoch 4/100
42000/42000 [==============================] - 5s 124us/sample - loss: 0.0214 - acc: 0.9928
Epoch 5/100
42000/42000 [==============================] - 5s 124us/sample - loss: 0.0166 - acc: 0.9950
Epoch 6/100
42000/42000 [==============================] - 5s 125us/sample - loss: 0.0112 - acc: 0.9960
Epoch 7/100
42000/42000 [==============================] - 5s 124us/sample - loss: 0.0114 - acc: 0.9963
Epoch 8/100
42000/42000 [==============================] - 5s 124us/sample - loss: 0.0080 - acc: 0.9977
Epoch 9/100
42000/42000 [==============================] - 5s 124us/sample - loss: 0.0087 - acc: 0.9970
Epoch 10/100
42000/42000 [=========================

In [40]:
#Predicting values from the test set
labels = my_model.predict(test_X)

In [41]:
sample_submission=pd.read_csv("digit-recognizer/sample_submission.csv")

In [ ]:
#Creating the necessary csv file for submissions -->

In [42]:
ImageId = test.index.values+1

In [43]:
labels = np.argmax(labels,axis=1)

In [44]:
data = {"ImageID":ImageId, "Label":labels}
df = pd.DataFrame(data=data)

In [45]:
df.to_csv("submission.csv", index=False)